In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers gradio

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition",
               "openai/whisper-large-v3",
               torch_dtype=torch.float16,
               device="cuda:0")

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
pipe("https://cdn-media.huggingface.co/speech_samples/sample1.flac")


{'text': " going along slushy country roads and speaking to damp audiences in draughty schoolrooms day after day for a fortnight he'll have to put in an appearance at some place of worship on sunday morning and he can come to us immediately afterwards"}

In [ ]:
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    return_all_scores=True
)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
import gradio as gr

def transcribe(audio_file=None, task="transcribe"):
    if audio_file is None:
        return "Please upload or record an audio file."

    # Check if the audio file is in bytes format (drag-and-drop file)
    if isinstance(audio_file, bytes):
        text = pipe(audio_file, generate_kwargs={"task": task}, return_timestamps=True)["text"]
    else:
        # Handle the case where the file is uploaded using the file uploader
        text = pipe(audio_file.name, generate_kwargs={"task": task}, return_timestamps=True)["text"]

    return text

# Create a Gradio interface with a File component for audio file upload
demo = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.File(type="binary", label="Upload Audio File"),
        gr.Radio(["transcribe", "translate"], label="Task"),
    ],
    outputs="text",
    title="Translate your Audio with Outlines",
    description=(
        "Transcribe long-form microphone or audio inputs with the click of a button!"
    ),
    allow_flagging="never",
    live=True,  # Add this line if you want to see the interface directly in the notebook
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://610489efef5d7a90f6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

def transcribe(audio_file=None, task="transcribe"):
    if audio_file is None:
        return "Please upload or record an audio file."

    # Check if the audio file is in bytes format (drag-and-drop file)
    if isinstance(audio_file, bytes):
        text = pipe(audio_file, generate_kwargs={"task": task}, return_timestamps=True)["text"]
    else:
        # Handle the case where the file is uploaded using the file uploader
        text = pipe(audio_file.name, generate_kwargs={"task": task}, return_timestamps=True)["text"]

    # Classify the transcribed text emotion
    emotion_scores = classifier(text)[0]
    most_likely_emotion = max(emotion_scores.items(), key=lambda x: x[1])[0]

    return {
        "text": text,
        "emotion": most_likely_emotion,
    }

# Create a Gradio interface with a File component for audio file upload
demo = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.File(type="binary", label="Upload Audio File"),
        gr.Radio(["transcribe", "translate"], label="Task"),
    ],
    outputs=["text", gr.Box(label="Emotion")],
    title="Translate your Audio with Emotions",
    description=(
        "Transcribe long-form microphone or audio inputs with the click of a button! Emotions will be displayed for the transcribed text."
    ),
    allow_flagging="never",
    live=True,  # Add this line if you want to see the interface directly in the notebook
)

demo.launch()




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


AttributeError: ignored